In [1]:
import os
from shared_lib.spark import get_spark_session

symbol = "ADAUSDT"
landing_date = "2025-09-27"
data_lake_bucket=os.getenv("DATA_LAKE_BUCKET", "your-data-lake-bucket")
iceberg_lock_table=os.getenv("ICEBERG_LOCK_TABLE", "your_iceberg_lock_table")
transform_db=os.getenv("TRANSFORM_DB", "your_transform_db")
spark = get_spark_session(app_name="dataops-crypto-cloud", master=True, local=True, jars=True, glue=True, iceberg_lock_table=iceberg_lock_table)

:: loading settings :: url = jar:file:/Users/anhtu/.pyenv/versions/3.11.11/envs/spark/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/anhtu/.ivy2/cache
The jars for the packages stored in: /Users/anhtu/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.iceberg#iceberg-aws-bundle added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ff1ca63e-9805-46aa-89fb-fdecb1668d92;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.7.1 in central
	found org.apache.iceberg#iceberg-aws-bundle;1.7.1 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 102ms :: artifacts dl 4ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.apache.iceberg#iceberg-aws-bundle;1.7.1 from cent

In [4]:
from shared_lib.spark import database_exists

if database_exists(spark, transform_db):
    print("Database exists")
else:
    print("Database does not exist")

Database exists


In [6]:
spark.sql(f"show tables in {transform_db}").show(truncate=False)

+------------------------------------------+-----------+-----------+
|namespace                                 |tableName  |isTemporary|
+------------------------------------------+-----------+-----------+
|crypto_cloud_dev_650251698703_transform_db|aggtrades  |false      |
|crypto_cloud_dev_650251698703_transform_db|klines     |false      |
|crypto_cloud_dev_650251698703_transform_db|pattern_two|false      |
+------------------------------------------+-----------+-----------+



In [7]:
spark.sql(f"select * from {transform_db}.pattern_two where pattern is not null").show()

+--------+-------------------+----------------+----------+----------+---------+-----------+--------+----------------+------------+-------+------+------+---------+-----------------+
|group_id|         group_date|       open_time|open_price|high_price|low_price|close_price|  volume|      close_time|landing_date| symbol|  ema7| ema20|    trend|          pattern|
+--------+-------------------+----------------+----------+----------+---------+-----------+--------+----------------+------------+-------+------+------+---------+-----------------+
| 1954413|2025-09-27 11:15:00|1758971700405060|    0.7837|    0.7853|   0.7837|      0.785|121594.4|1758972591323187|  2025-09-27|ADAUSDT|0.7835| 0.784|downtrend|bullish engulfing|
| 1954445|2025-09-27 19:15:00|1759000502967967|    0.7803|    0.7823|     0.78|     0.7821| 95233.0|1759001384883232|  2025-09-27|ADAUSDT|0.7808|0.7817|downtrend|bullish engulfing|
+--------+-------------------+----------------+----------+----------+---------+-----------+----